In [23]:
import re
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

## Extracting data

In [24]:
%%time
df = pd.read_csv('chess_games.csv', nrows=500000,
                usecols=['Event', 'Result', 'UTCDate', 'Opening', 'Termination', 'AN']) # chunksize=100000, index_col=''
print(f"{len(df)} rows was extracted")

500000 rows was extracted
CPU times: total: 5.48 s
Wall time: 6.84 s


In [25]:
df.head(3)

,Event,Result,UTCDate,Opening,Termination,AN
0,Classical,1-0,2016.06.30,Slav Defense,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,0-1,2016.06.30,King's Pawn Opening: 2.b3,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,1-0,2016.06.30,Scandinavian Defense: Mieses-Kotroc Variation,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....


## Transform data

### Filter dataframe

#### Get rid of ambiguous match results and matches that was abandoned or have rules infractions

In [26]:
filt = (df['Result'] != '*') & (df['Termination'] != 'Abandoned') & (df['Termination'] != 'Rules infraction')
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

1057 rows was removed
498943 rows left


#### Remove short matches (~less than 10 moves)

In [27]:
filt = df["AN"].apply(len) > 50
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

5689 rows was removed
493254 rows left


#### Get rid of unstructured string values in "AN" column

In [28]:
filt = ~df['AN'].str.contains('\[%eval')
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

63866 rows was removed
429388 rows left


#### Remove unpopular openings (less than 1500 matches)

In [7]:
# value_counts = df.stack().value_counts() # Entire DataFrame 
# to_remove = value_counts[value_counts <= 1500].index
# df.replace(to_remove, np.nan, inplace=True)

In [11]:
vc = df['Opening'].value_counts()
vals_to_remove = vc[vc < 1500].index.values
df['Opening'].loc[df['Opening'].isin(vals_to_remove)] = '111'
filt = df['Opening'] != '111'
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

255831 rows was removed
173557 rows left


### Expand and rearrange dataframe

#### Reset index

In [29]:
df = df.reset_index(drop=True)

#### Create column "Tournament"

In [30]:
df['Tournament'] = df['Event'].str.contains('tournament')

#### Rearrange column "Event"

In [31]:
df['Event'] = df['Event'].map({' Classical ': 'Classical',
                               ' Blitz ': 'Blitz',
                               ' Blitz tournament ': 'Blitz',
                               ' Correspondence ': 'Classical',
                               ' Classical tournament ': 'Classical',
                               ' Bullet tournament ': 'Bullet',
                               ' Bullet ': 'Bullet',
                               'Blitz tournament ': 'Blitz',
                               'Bullet ': 'Bullet',
                               'Classical ': 'Classical',
                               'Blitz ': 'Blitz'
                               })

#### Change columns datatype

In [32]:
df['Event'] = df['Event'].astype('category')
df['Result'] = df['Result'].astype('category')
df['Termination'] = df['Termination'].astype('category')
df['UTCDate'] = pd.to_datetime(df['UTCDate'])

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429388 entries, 0 to 429387
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Event        429388 non-null  category      
 1   Result       429388 non-null  category      
 2   UTCDate      429388 non-null  datetime64[ns]
 3   Opening      429388 non-null  object        
 4   Termination  429388 non-null  category      
 5   AN           429388 non-null  object        
 6   Tournament   429388 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), object(2)
memory usage: 11.5+ MB


#### Create moves_total from AN column

In [34]:
def delete_result_from_an(move: str) -> str:
    '''Delete result mark in the last move.'''
    move = re.sub(' 1-0',     '', move)
    move = re.sub(' 0-1',     '', move)
    move = re.sub(' 1/2-1/2', '', move)
    move = re.sub(' \*',      '', move)
    return move

In [14]:
def populate_moves_total(row: list[str]) -> None:
    '''
    Split each move (string) into a tuple.
    Tuple format: (match_id, move_num, white_move, black_move)
    Match_id is foreign key of the main dataframe (index column).
    '''
    for move in row:
        split_list = move.split(' ')
        tuple_move = tuple([0] + [int(split_list[0])] + split_list[1:])

        if len(tuple_move) == 3:
            tuple_move = tuple_move + (None, ) # in case of last move ('0', '38', 'Rfd1', None)

        moves_total.append(tuple_move)

In [15]:
def create_moves_total(row: str) -> None:
    '''Separate each move.'''
    match_moves = []
    match_moves.append(row)
    sep_num = 2
    while True:  # parse as many moves as possible
        separator = str(sep_num)+'.'
        if separator in match_moves[-1]:  # if the next move is found
            match_moves = match_moves[0:-1] + match_moves[-1].split(' '+separator) # concatenate all previous moves and remainder
            match_moves[-1] = str(sep_num) + match_moves[-1]
            sep_num+=1
        else:  # if the next move is not found
            match_moves[-1] = delete_result_from_an(match_moves[-1])
            match_moves[0] =  re.sub('1.', '1', match_moves[0])
            break
    
    populate_moves_total(match_moves)

In [16]:
%%time
%%capture
moves_total: list[tuple] = [] # template for data frame
df["AN"].apply(create_moves_total)

# 429388 rows = 
# CPU times: total: 1min 1s
# Wall time: 1min 7s
# 59.8 s ± 3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

CPU times: total: 1min 10s
Wall time: 1min 16s


In [17]:
len(moves_total)

14822951

### moves_total1

In [35]:
def populate_moves_total1(row: list[str], ind: int) -> None:
    '''
    Split each move (string) into a tuple.
    Tuple format: (match_id, move_num, white_move, black_move)
    Match_id is foreign key of the main dataframe (index column).
    '''
    for move in row:
        split_list = move.split(' ')
        tuple_move = tuple([ind] + [int(split_list[0])] + split_list[1:])

        if len(tuple_move) == 3:
            tuple_move = tuple_move + (None, ) # in case of last move ('0', '38', 'Rfd1', None)

        moves_total1.append(tuple_move)

In [36]:
def create_moves_total1(row) -> None:
    '''Separate each move.'''
    match_moves = []
    match_moves.append(row["AN"])
    sep_num = 2
    while True:  # parse as many moves as possible
        separator = str(sep_num)+'.'
        if separator in match_moves[-1]:  # if the next move is found
            match_moves = match_moves[0:-1] + match_moves[-1].split(' '+separator) # concatenate all previous moves and remainder
            match_moves[-1] = str(sep_num) + match_moves[-1]
            sep_num+=1
        else:  # if the next move is not found
            match_moves[-1] = delete_result_from_an(match_moves[-1])
            match_moves[0] =  re.sub('1.', '1', match_moves[0])
            break
    
    populate_moves_total1(match_moves, row.name)

In [37]:
# df_an = df["AN"].copy()
# df_an = pd.DataFrame(df_an)

In [38]:
%%time
%%capture
moves_total1: list[tuple] = [] # template for data frame
df.apply(create_moves_total1, axis = 1)
# 429388 rows = 1min 1s ± 1.74 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

CPU times: total: 1min 23s
Wall time: 1min 43s


In [75]:
# moves_total1 == moves_total

True

#### Convert moves_total to df_moves

In [41]:
%%time
df_moves = pd.DataFrame(moves_total1, columns=['match_id', 'move_num', 'white_move', 'black_move'])

CPU times: total: 12.3 s
Wall time: 12.6 s


In [42]:
df_moves

,match_id,move_num,white_move,black_move
0,0,1,d4,d5
1,0,2,c4,c6
2,0,3,e3,a6
3,0,4,Nf3,e5
4,0,5,cxd5,e4
...,...,...,...,...
14822946,429387,43,bxc5,Kg4
14822947,429387,44,Be5,Kf5
14822948,429387,45,Bg3,Kxe6
14822949,429387,46,Kc3,Kd5


In [43]:
df_moves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14822951 entries, 0 to 14822950
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   match_id    int64 
 1   move_num    int64 
 2   white_move  object
 3   black_move  object
dtypes: int64(2), object(2)
memory usage: 452.4+ MB


#### Drop AN column

In [238]:
df = df.drop('AN', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429388 entries, 0 to 429387
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Event        429388 non-null  category      
 1   Result       429388 non-null  category      
 2   UTCDate      429388 non-null  datetime64[ns]
 3   Opening      429388 non-null  object        
 4   Termination  429388 non-null  category      
 5   Tournament   429388 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), object(1)
memory usage: 8.2+ MB


## Draft

In [125]:
df_moves['move_num'].max()

290

In [129]:
df_moves.query('move_num >= 290')

,match_id,move_num,white_move,black_move
13634638,0,290,Kf2,None


In [ ]:
# %%time
# for i, row in df.iterrows():
#     df.at[i, 'Event'] = df.at[i, 'Event']

In [ ]:
# %%time
# for i in df.index:
#     df['Event'].iloc[i] = df['Event'].iloc[i]

In [188]:
length = 0
for row in range(len(df)):
    length = length + len(df['AN'][row])
length

35474522

In [125]:
# df_moves = pd.DataFrame({'match_id': pd.Series(dtype='int16'),
#                          'move_num': pd.Series(dtype='int16'),
#                          'white_move': pd.Series(dtype='str'),
#                          'black_move': pd.Series(dtype='str')})